# JupyterDash
The `jupyter-dash` package makes it easy to develop Plotly Dash apps from the Jupyter Notebook and JupyterLab.

Just replace the standard `dash.Dash` class with the `jupyter_dash.JupyterDash` subclass.


## Before you start coding:
* Design your dashboard! Think about the layout and what Dash Core Components you'll want to use
    * dccs documentation: https://dash.plotly.com/dash-core-components
* If possible, think about what components will interact with each other. This'll make coming up with Callbacks a lot easier


In [1]:
from jupyter_dash import JupyterDash

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd

When running in JupyterHub or Binder, call the `infer_jupyter_config` function to detect the proxy configuration.

In [3]:
# JupyterDash.infer_jupyter_proxy_config()

Load and preprocess data

In [4]:
df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')
available_indicators = df['Indicator Name'].unique()

In [5]:
df

,Country Name,Indicator Name,Year,Value
0,Arab World,"Agriculture, value added (% of GDP)",1962,NaN
1,Arab World,CO2 emissions (metric tons per capita),1962,0.760996
2,Arab World,Domestic credit provided by financial sector (...,1962,18.168690
3,Arab World,Electric power consumption (kWh per capita),1962,NaN
4,Arab World,Energy use (kg of oil equivalent per capita),1962,NaN
...,...,...,...,...
36955,Zimbabwe,"Industry, value added (% of GDP)",2007,33.074953
36956,Zimbabwe,"Inflation, GDP deflator (annual %)",2007,0.894887
36957,Zimbabwe,"Life expectancy at birth, total (years)",2007,44.177756
36958,Zimbabwe,Population density (people per sq. km of land ...,2007,34.374559


Construct the app and callbacks

In [6]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

# Create server variable with Flask server object for use with gunicorn
server = app.server

########## THIS IS YOUR APP'S LAYOUT ##########

app.layout = html.Div([
    # this is the top grey bar
    html.Div([
        # left side
        html.Div([ # elements within a div by default appear top to bottom 
            # that's why right under the dropdown you have the Linear/Log selection 
            dcc.Dropdown(
                id='crossfilter-xaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Fertility rate, total (births per woman)'
            ),
            dcc.RadioItems(
                id='crossfilter-xaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log', 'Exponent']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}), 
        # this line pushes everything in this div to take up 49% of the width (left column)
        
        # right side
        html.Div([
            dcc.Dropdown(
                id='crossfilter-yaxis-column',
                options=[{'label': i, 'value': i} for i in available_indicators],
                value='Life expectancy at birth, total (years)'
            ),
            dcc.RadioItems(
                id='crossfilter-yaxis-type',
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'}) # leans this div to the right
    ], style={ # this is the style for the entire top grey bar div
        'borderBottom': 'thin lightgrey solid',
        'backgroundColor': 'rgb(250, 250, 250)',
        'padding': '10px 5px'
    }),

    # this div houses the three graphs
    html.Div([
        dcc.Graph( # these are like "placeholders" for the graphs that you'll later update w callbacks
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]} # initial value
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    
    
    html.Div([
        dcc.Graph(id='x-time-series'), # more placeholders
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider( # slider div
        id='crossfilter-year--slider',
        min=df['Year'].min(),
        max=df['Year'].max(),
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()},
        step=None
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


# IMPT TO REMEMBER FOR CALLBACKS
# - the output is the ID of the placeholder graph from the layout
# - the number AND order of inputs matches the arguments in the function defined below
# - names of arguments don't have to match
# - the function returns the graph

### SCATTERPLOT CALLBACK ###
@app.callback(
    dash.dependencies.Output('crossfilter-indicator-scatter', 'figure'),
    [dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value'),
     dash.dependencies.Input('crossfilter-year--slider', 'value')])
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    return {
        'data': [dict(
            x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            text=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'],
            mode='markers',
            marker={
                'size': 25,
                'opacity': 0.7,
                'color': 'orange',
                'line': {'width': 2, 'color': 'purple'}
            }
        )],
        'layout': dict(
            xaxis={
                'title': xaxis_column_name,
                'type': 'linear' if xaxis_type == 'Linear' else 'log'
            },
            yaxis={
                'title': yaxis_column_name,
                'type': 'linear' if yaxis_type == 'Linear' else 'log'
            },
            margin={'l': 40, 'b': 30, 't': 10, 'r': 0},
            height=450,
            hovermode='closest'
        )
    }

# additional helper function for the line graphs on the right
def create_time_series(dff, axis_type, title):
    return {
        'data': [dict(
            x=dff['Year'],
            y=dff['Value'],
            mode='lines+markers'
        )],
        'layout': {
            'height': 225,
            'margin': {'l': 20, 'b': 30, 'r': 10, 't': 10},
            'annotations': [{
                'x': 0, 'y': 0.85, 'xanchor': 'left', 'yanchor': 'bottom',
                'xref': 'paper', 'yref': 'paper', 'showarrow': False,
                'align': 'left', 'bgcolor': 'rgba(255, 255, 255, 0.5)',
                'text': title
            }],
            'yaxis': {'type': 'linear' if axis_type == 'Linear' else 'log'},
            'xaxis': {'showgrid': False}
        }
    }

### CALLBACKS FOR THE LINE GRAPHS ###
@app.callback(
    dash.dependencies.Output('x-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-xaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-xaxis-type', 'value')])
def update_y_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@app.callback(
    dash.dependencies.Output('y-time-series', 'figure'),
    [dash.dependencies.Input('crossfilter-indicator-scatter', 'hoverData'),
     dash.dependencies.Input('crossfilter-yaxis-column', 'value'),
     dash.dependencies.Input('crossfilter-yaxis-type', 'value')])
def update_x_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)

Serve the app using `run_server`.  Unlike the standard `Dash.run_server` method, the `JupyterDash.run_server` method doesn't block execution of the notebook. It serves the app in a background thread, making it possible to run other notebook calculations while the app is running.

This makes it possible to iterativly update the app without rerunning the potentially expensive data processing steps.

In [12]:
# use this to run in a new browser tab/window
app.run_server()

/usr/local/lib/python3.9/site-packages/jupyter_dash/jupyter_app.py:139: UserWarning: The 'environ['werkzeug.server.shutdown']' function is deprecated and will be removed in Werkzeug 2.1.
  func()


Dash app running on http://127.0.0.1:8050/


By default, `run_server` displays a URL that you can click on to open the app in a browser tab. The `mode` argument to `run_server` can be used to change this behavior.  Setting `mode="inline"` will display the app directly in the notebook output cell.

In [7]:
app.run_server(mode="inline")

When running in JupyterLab, with the `jupyterlab-dash` extension, setting `mode="jupyterlab"` will open the app in a tab in JupyterLab.

```python
app.run_server(mode="jupyterlab")
```